In [1]:
import cv2
import numpy as np
import pandas as pd
import bundle_adjustment as ba
import reconstruction as rec
import matching as matcher
import os
import glob

base_path = os.getcwd()

In [2]:
def get_images(base_path, dataset, img_format, use_n_imgs=-1):
    images = []
    images_paths = sorted(
        glob.glob(
            os.path.join(base_path, "datasets", dataset) + "/*." + img_format,
            recursive=True,
        )
    )
    if not use_n_imgs==-1 and use_n_imgs<=len(images_paths):
        images_paths=images_paths[:use_n_imgs]
        
    for images_path in images_paths:
        try:
            images.append(cv2.imread(images_path, cv2.IMREAD_GRAYSCALE))
        except:
            raise ValueError(
                f"Need to pass in valid imgset name! Tried to read {images_path}"
            )

    return images

In [3]:
n_imgs = 46  # 46 if imgset = 'templering', 49 if imgset = 'Viking'
imgset = "templeRing"
K = np.matrix("1520.40 0.00 302.32; 0.00 1525.90 246.87; 0.00 0.00 1.00")

# imgset = "dino"
# n_imgs = 46
# K = np.matrix(
#     "3310.400000 0.000000 316.730000; 0.000000 3325.500000 200.550000; 0.000000 0.000000 1.000000"
# )
images = get_images(base_path, imgset, "png", n_imgs)
assert len(images) == n_imgs

In [4]:
keypoints, descriptors = matcher.get_sift_features(images)
bf_matcher = cv2.BFMatcher(cv2.NORM_L1)
matches = matcher.find_matches(bf_matcher, keypoints, descriptors, 0.65)
print('num_matches before outlier removal:', matcher.num_matches(matches))
matcher.print_num_img_pairs(matches)

matches = matcher.remove_outliers(matches, keypoints)
print("After outlier removal:")
matcher.print_num_img_pairs(matches)
img_adjacency, list_of_img_pairs = matcher.create_img_adjacency_matrix(n_imgs, matches)

[ WARN:0@2.358] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


num_matches before outlier removal: 49934
Number of img pairs is 1011 out of possible 1035
After outlier removal:
Number of img pairs is 226 out of possible 1035


In [5]:
### This cell initializes the reconstruction
best_pair = rec.best_img_pair(img_adjacency, matches, keypoints, K, top_x_perc=0.2)
R0, t0, R1, t1, points3d_with_views = rec.initialize_reconstruction(keypoints, matches, K, best_pair[0], best_pair[1])

R_mats = {best_pair[0]: R0, best_pair[1]: R1}
t_vecs = {best_pair[0]: t0, best_pair[1]: t1}

resected_imgs = [best_pair[0], best_pair[1]]
unresected_imgs = [i for i in range(len(images)) if i not in resected_imgs] 
print('initial image pair:', resected_imgs)
avg_err = 0

Triangulating: 372 points.
initial image pair: [23, 25]


In [6]:
### This cell grows and refines the reconstruction 
BA_chkpts = [3,4,5,6] + [int(6*(1.34**i)) for i in range(25)]
while len(unresected_imgs) > 0:
    resected_idx, unresected_idx, prepend = rec.next_img_pair_to_grow_reconstruction(n_imgs, best_pair, resected_imgs, unresected_imgs, img_adjacency)
    points3d_with_views, pts3d_for_pnp, pts2d_for_pnp, triangulation_status = rec.get_correspondences_for_pnp(resected_idx, unresected_idx, points3d_with_views, matches, keypoints)
    if len(pts3d_for_pnp) < 12:
        print(f"{len(pts3d_for_pnp)} is too few correspondences for pnp. Skipping imgs resected:{resected_idx} and unresected:{unresected_idx}")
        print(f"Currently resected imgs: {resected_imgs}, unresected: {unresected_imgs}")
        continue

    R_res = R_mats[resected_idx]
    t_res = t_vecs[resected_idx]
    print(f"Unresected image: {unresected_idx}, resected: {resected_idx}")
    R_new, t_new = rec.do_pnp(pts3d_for_pnp, pts2d_for_pnp, K)
    R_mats[unresected_idx] = R_new
    t_vecs[unresected_idx] = t_new
    if prepend: resected_imgs.insert(0, unresected_idx)
    else: resected_imgs.append(unresected_idx)
    unresected_imgs.remove(unresected_idx)
    pnp_errors, projpts, avg_err, perc_inliers = rec.test_reproj_pnp_points(pts3d_for_pnp, pts2d_for_pnp, R_new, t_new, K)
    print(f"Average error of reprojecting points used to resect image {unresected_idx} back onto it is: {avg_err}")
    print(f"Fraction of Pnp inliers: {perc_inliers} num pts used in Pnp: {len(pnp_errors)}")
    
    if resected_idx < unresected_idx:
        kpts1, kpts2, kpts1_idxs, kpts2_idxs = rec.get_aligned_kpts(resected_idx, unresected_idx, keypoints, matches, mask=triangulation_status)
        if np.sum(triangulation_status) > 0: #at least 1 point needs to be triangulated
            points3d_with_views, tri_errors, avg_tri_err_l, avg_tri_err_r = rec.triangulate_points_and_reproject(R_res, t_res, R_new, t_new, K, points3d_with_views, resected_idx, unresected_idx, kpts1, kpts2, kpts1_idxs, kpts2_idxs, reproject=True)
    else:
        kpts1, kpts2, kpts1_idxs, kpts2_idxs = rec.get_aligned_kpts(unresected_idx, resected_idx, keypoints, matches, mask=triangulation_status)
        if np.sum(triangulation_status) > 0: #at least 1 point needs to be triangulated
            points3d_with_views, tri_errors, avg_tri_err_l, avg_tri_err_r = rec.triangulate_points_and_reproject(R_new, t_new, R_res, t_res, K, points3d_with_views, unresected_idx, resected_idx, kpts1, kpts2, kpts1_idxs, kpts2_idxs, reproject=True)
    
    if 0.8 < perc_inliers < 0.95 or 5 < avg_tri_err_l < 10 or 5 < avg_tri_err_r < 10: 
        #If % of inlers from Pnp is too low or triangulation error on either image is too high, bundle adjust
        points3d_with_views, R_mats, t_vecs = ba.do_BA(points3d_with_views, R_mats, t_vecs, resected_imgs, keypoints, K, ftol=1e0)
        
    if len(resected_imgs) in BA_chkpts or len(unresected_imgs) == 0 or perc_inliers <= 0.8 or avg_tri_err_l >= 10 or avg_tri_err_r >= 10:
        #If % of inlers from Pnp is very low or triangulation error on either image is very high, bundle adjust with stricter tolerance
        points3d_with_views, R_mats, t_vecs = ba.do_BA(points3d_with_views, R_mats, t_vecs, resected_imgs, keypoints, K, ftol=1e-1)
    
    av = 0
    for im in resected_imgs:
        p3d, p2d, avg_error, errors = rec.get_reproj_errors(im, points3d_with_views, R_mats[im], t_vecs[im], K, keypoints, distCoeffs=np.array([]))
        print(f'Average reprojection error on image {im} is {avg_error} pixels')
        av += avg_error
    av = av/len(resected_imgs)
    print(f'Average reprojection error across all {len(resected_imgs)} resected images is {av} pixels')

Unresected image: 24, resected: 23
rvec: [[-0.15227498]
 [-0.00463762]
 [ 0.01802589]] 

tvec: [[ 0.01969175]
 [-0.52677383]
 [ 0.04154978]]
Average error of reprojecting points used to resect image 24 back onto it is: 0.13236054922174492
Fraction of Pnp inliers: 1.0 num pts used in Pnp: 311
Triangulating: 230 points.
Average reprojection error for just-triangulated points on image 23 is: 0.06741888890888598 pixels.
Average reprojection error for just-triangulated points on image 24 is: 0.06789456378644983 pixels.
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.6610e+01                                    2.49e+04    
       1              3         4.0421e+01      6.19e+00       7.36e-01       3.98e+04    
       2              4         3.2396e+01      8.02e+00       3.00e-01       1.98e+04    
       3              5         3.2099e+01      2.97e-01       7.18e-01       3.22e+04    
       4              6  

In [7]:
### This cell visualizes the pointcloud
num_voxels = 200 #Set to 100 for faster visualization, 200 for higher resolution.
x, y, z = [], [], []
for pt3 in points3d_with_views:
    if abs(pt3.point3d[0][0]) + abs(pt3.point3d[0][1]) + abs(pt3.point3d[0][2]) < 100:
        x.append(pt3.point3d[0][0])
        y.append(pt3.point3d[0][1])
        z.append(pt3.point3d[0][2])
vpoints = list(zip(x,y,z))
vpoints = np.array(vpoints)
vpoints_df = pd.DataFrame(data=vpoints, index=[f"{i}" for i in range(vpoints.shape[0])], columns=["x", "y","z"])
# cloud = pyntcloud.PyntCloud(vpoints_df)
# cloud.add_structure('voxelgrid', n_x=num_voxels,n_y=num_voxels,n_z=num_voxels)
# cloud.structures[f'V([{num_voxels}, {num_voxels}, {num_voxels}],[None, None, None],True)'].plot(d=3)

In [8]:
import open3d as o3d
def visualize_sfm_open3d(points_3d):
    """
    Visualizes a 3D point cloud obtained from Structure from Motion (SfM) using Open3D.

    Parameters:
    points_3d (numpy.ndarray): A 2D NumPy array representing the 3D points. Each row contains the (x, y, z) coordinates of a point.

    Returns:
    None. The function displays the 3D point cloud in a new window using Open3D.
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_3d)
    o3d.visualization.draw_geometries([pcd], window_name="SfM 3D Reconstruction")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
visualize_sfm_open3d(vpoints)